<a href="https://colab.research.google.com/github/patharepremkumar/Restaurent-recommendation-system/blob/main/LightFM_hybrid_restaurent_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install lightfm

  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808325 sha256=75473f0567e3ab6153536721b507f6fb86f46b0c5824997c20b007ff65feb988
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [33]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score
import numpy as np
from lightfm.cross_validation import random_train_test_split
import os
from scipy.sparse import csr_matrix
pd.set_option('display.max_columns', None)
import json

In [34]:
restaurent_df = pd.read_csv('/content/processed_restaurent_details.csv')
users_df = pd.read_csv('/content/processed_user_details.csv')
review_df = pd.read_csv('/content/processed_review_datasets.csv')

# prompt: rename _id column from user_df dataset

users_df.rename(columns={"_id": "user_id"}, inplace=True)
restaurent_df.rename(columns={"_id": "restaurant_id"}, inplace=True)

In [35]:
restaurent_df.shape,review_df.shape

((647, 249), (1577, 28))

In [36]:
review_df[review_df['user_id']=='60b75e256dcadb0291ef799c']

,Unnamed: 0,_id,likes,dislikes,skiped,status,isUser,type,review,rating,request_time,user_id,vendor_id,restaurant_id,review_id,comment,createdAt,updatedAt,__v,event_request_id,event_id,order_id,diner_id,diner_request_id,item_id,transactional_id,experienceId,expRequestId


In [37]:
restaurent_df.head(2)
restaurent_df['restaurant_name'].isnull().sum(),restaurent_df.shape

(258, (647, 249))

In [38]:
rating_dataset = review_df[['user_id','restaurant_id','rating']]
user_dataset = users_df[['user_id', 'are_you_over_21','allergies_flag','gender']]
restaurent_dataset = restaurent_df[['restaurant_id', 'allergic_information','serve_liquor','hygiene_information','covid_compliance']]

In [39]:
# prompt: code to filter out nan value from rating column of review_df dataframe

rating_dataset = rating_dataset[pd.notnull(rating_dataset['rating'])]
rating_dataset = rating_dataset[pd.notnull(rating_dataset['restaurant_id'])]


In [40]:
# prompt: code to convert review_df dataframe user_id column from string to dictionary

rating_dataset['user_id'] = rating_dataset['user_id'].apply(lambda x: json.loads(x.replace("'", '"'))['$oid'])
rating_dataset['restaurant_id'] = rating_dataset['restaurant_id'].apply(lambda x: json.loads(x.replace("'", '"'))['$oid'])


In [41]:
rating_dataset.isnull().sum()

user_id          0
restaurant_id    0
rating           0
dtype: int64

In [42]:
restaurent_dataset[restaurent_dataset['restaurant_id']=='61e9354f3d9df721dca15168']

,restaurant_id,allergic_information,serve_liquor,hygiene_information,covid_compliance
167,61e9354f3d9df721dca15168,False,False,False,False


In [43]:
# prompt: how to remove userid list values from rating dataset

user_id_list = ['60b75e256dcadb0291ef799c']
rating_dataset = rating_dataset[~rating_dataset['user_id'].isin(user_id_list)]


In [44]:
u_cols = ['user_id', 'are_you_over_21','allergies_flag','gender']
m_cols = ['restaurant_id', 'allergic_information','serve_liquor','hygiene_information','covid_compliance']
r_cols = ['user_id', 'restaurant_id', 'rating']

In [45]:
user_dataset.head(2)

,user_id,are_you_over_21,allergies_flag,gender
0,60b720446dcadb0291ef772e,True,False,Male
1,60b721a36dcadb0291ef7736,False,False,NaN


In [46]:
# prompt: pandas pd.get_dummies code to convert ['are_you_over_21','allergies_flag','gender'] these column from user_dataset dataframe

final_user_dataset = pd.get_dummies(user_dataset, columns=['are_you_over_21','allergies_flag','gender'])
final_user_dataset.head()

final_restaurent_dataset = pd.get_dummies(restaurent_dataset,columns=['allergic_information','serve_liquor','hygiene_information','covid_compliance'])
final_restaurent_dataset.head()


,restaurant_id,allergic_information_False,allergic_information_True,serve_liquor_False,serve_liquor_True,hygiene_information_False,covid_compliance_False
0,60b5c03a6dcadb0291ef6cfe,True,False,True,False,True,True
1,60b664376dcadb0291ef6e7f,False,True,False,True,True,True
2,60ba083f6dcadb0291ef8864,True,False,True,False,True,True
3,60bac83c6dcadb0291ef92e3,False,True,True,False,True,True
4,60bb36bb6dcadb0291ef9797,True,False,False,True,True,True


In [47]:
rating_dataset = rating_dataset[rating_dataset['user_id'].isin(final_user_dataset['user_id'])]

In [48]:
item_features = final_restaurent_dataset
item_features.shape

(647, 7)

In [49]:
user_features_col = final_user_dataset.drop(columns =['user_id']).columns.values
user_feat = final_user_dataset.drop(columns =['user_id']).to_dict(orient='records')

item_features_col = final_restaurent_dataset.drop(columns=['restaurant_id']).columns.values
item_feat = final_restaurent_dataset.drop(columns =['restaurant_id']).to_dict(orient='records')

In [50]:
len(item_feat)

647

In [51]:
dataset = Dataset()
dataset.fit(users=[x for x in final_user_dataset['user_id']], items=[x for x in final_restaurent_dataset['restaurant_id']], item_features=item_features_col, user_features=user_features_col)

num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 1128, num_items 647.


In [55]:
dataset.user_features_shape()

(1128, 1135)

In [57]:
item_features = dataset.build_item_features((x,y) for x,y in zip(final_restaurent_dataset['restaurant_id'],item_feat))
user_features = dataset.build_user_features((x,y) for x,y in zip(final_user_dataset['user_id'],user_feat))

In [58]:
(interactions, weights) = dataset.build_interactions((x, y)
                                                      for x,y in zip(rating_dataset['user_id'], rating_dataset['restaurant_id']))

In [61]:
# look at item feature mapping
a = dataset.mapping()[3]
list(a.items())[0:10]

[('60b5c03a6dcadb0291ef6cfe', 0),
 ('60b664376dcadb0291ef6e7f', 1),
 ('60ba083f6dcadb0291ef8864', 2),
 ('60bac83c6dcadb0291ef92e3', 3),
 ('60bb36bb6dcadb0291ef9797', 4),
 ('60d9668d52f4960844d62626', 5),
 ('60e190281ba27d6c59733beb', 6),
 ('60e2c8d01953de196e11a7ba', 7),
 ('60e2ea50e6f61f1efdc23036', 8),
 ('60e3e7c6e6f61f1efdc250a1', 9)]

In [63]:
from scipy.sparse import csr_matrix, coo_matrix
# Convert the interaction matrix to CSR format
interactions = csr_matrix(interactions)
weights = csr_matrix(weights)

# Convert the CSR matrix back to COO format
#interactions = coo_matrix(interactions)
#weights = coo_matrix(weights)

In [65]:
train, test = random_train_test_split(interactions,test_percentage=0.2, random_state=np.random.RandomState(seed=42))
train_w, test_w = random_train_test_split(weights, test_percentage=0.2, random_state=np.random.RandomState(seed=42))

In [66]:
# Print the number of users and items in the test and train interactions.
print('Test interactions: users {}, items {}'.format(test.shape[0], test.shape[1]))
print('Train interactions: users {}, items {}'.format(train.shape[0], train.shape[1]))

# Check for overlapping interactions between test and train.
overlapping_interactions = test.multiply(train).nnz
print('Number of overlapping interactions: {}'.format(overlapping_interactions))

Test interactions: users 1128, items 647
Train interactions: users 1128, items 647
Number of overlapping interactions: 0


In [67]:
n_components = 30
loss = 'warp'
epoch = 30
num_thread = 4
model = LightFM(no_components= n_components, loss=loss, random_state = 1616)
model.fit(train,  user_features= user_features, item_features= item_features, epochs=epoch,num_threads = num_thread, sample_weight = train_w)

In [68]:
train_precision = precision_at_k(model, train, k=3,item_features=item_features, user_features=user_features).mean()
test_precision = precision_at_k(model, test,train_interactions=train, k=3,item_features=item_features, user_features=user_features).mean()

train_recall = recall_at_k(model, train, k=5,item_features=item_features, user_features=user_features).mean()
test_recall = recall_at_k(model, test,train_interactions=train, k=5,item_features=item_features, user_features=user_features).mean()

train_auc = auc_score(model, train,item_features=item_features, user_features=user_features).mean()
test_auc = auc_score(model, test, train_interactions=train,item_features=item_features, user_features=user_features).mean()

print('Precision: train %.2f' % (train_precision))
print('Precision: test %.2f' % (test_precision))

print('Recall: train %.2f' % (train_recall))
print('Recall: test %.2f' % (test_recall))

print('AUC: train %.2f' % (train_auc))
print('AUC: test %.2f' % (test_auc))

Precision: train 0.25
Precision: test 0.13
Recall: train 0.46
Recall: test 0.40
AUC: train 0.98
AUC: test 0.95


In [78]:
def similar_items(item_id, model, N=10, norm = True):
    item_bias ,item_representations = model.get_item_representations(features=item_features)

    # Cosine similarity
    scores = item_representations.dot(item_representations[item_id, :])
    item_norms = np.linalg.norm(item_representations, axis=1)

    if norm == True:
        scores /= item_norms
        best = np.argpartition(scores, -N)[-N:]
        similar = sorted(zip(best, scores[best]/ item_norms[item_id] ), key=lambda x: -x[1])
    else:
        best = np.argpartition(scores, -N)[-N:]
        similar = sorted(zip(best, scores[best] ), key=lambda x: -x[1])
    return similar

similar_item_list = similar_items(13, model)
similar_idx = [x[0] for x in similar_item_list ]


[(13, 1.0),
 (356, 0.9984212),
 (164, 0.9982804),
 (239, 0.99820095),
 (99, 0.99805737),
 (477, 0.99798125),
 (85, 0.99789745),
 (252, 0.9977286),
 (296, 0.9976978),
 (211, 0.99769545)]

In [234]:
known_positives_rating = rating_dataset[(rating_dataset['user_id']=='60dfa43e1ba27d6c59733859')].merge(final_restaurent_dataset, on = 'restaurant_id')
known_positives_rating[known_positives_rating['restaurant_id'].isin(top_items['restaurant_id'][0:10])]

known_positives_rating.sort_values(by=['rating'], ascending = False)

,user_id,restaurant_id,rating,allergic_information_False,allergic_information_True,serve_liquor_False,serve_liquor_True,hygiene_information_False,covid_compliance_False
0,60dfa43e1ba27d6c59733859,60b743606dcadb0291ef7869,3.0,False,True,False,True,True,True


## Similar item calculation from item features

In [84]:
def similar_items(item_id, model, N=5, norm = True):
    item_bias ,item_representations = model.get_item_representations(features=item_features)

    # Cosine similarity
    scores = item_representations.dot(item_representations[item_id, :])
    item_norms = np.linalg.norm(item_representations, axis=1)

    if norm == True:
        scores /= item_norms
        best = np.argpartition(scores, -N)[-N:]
        similar = sorted(zip(best, scores[best]/ item_norms[item_id] ), key=lambda x: -x[1])
    else:
        best = np.argpartition(scores, -N)[-N:]
        similar = sorted(zip(best, scores[best] ), key=lambda x: -x[1])
    return similar

similar_item_list = similar_items(4, model)
similar_idx = [x[0] for x in similar_item_list ]
restaurent_df.iloc[similar_idx][['restaurant_name','restaurant_description','city']]

,restaurant_name,restaurant_description,city
4,Krishna Veg,North indian restaurant\n,Pune
10,Global Punjab,"We created a sophisticated, warm and harmoniou...",Pune
129,Villa Casa - The Cult Cuisine,Trendy restaurant with a terrace specializing ...,Pune
61,China Grill,It is 170-seater posh fine dining two-level re...,Pune
576,Roccomama's,Amazing north indian restaurant,Pune Division


In [100]:
def similar_users(user_id, model, N=3, norm = True):
    user_bias ,user_representations = model.get_user_representations(features= user_features)

    # Cosine similarity
    scores = user_representations.dot(user_representations[user_id, :])
    item_norms = np.linalg.norm(user_representations, axis=1)

    if norm == True:
        scores /= item_norms
        best = np.argpartition(scores, -N)[-N:]
        similar = sorted(zip(best, scores[best] / item_norms[user_id]),
                    key=lambda x: -x[1])
    else:
        best = np.argpartition(scores, -N)[-N:]
        similar = sorted(zip(best, scores[best]),
                    key=lambda x: -x[1])
    return similar

similar_item_list = similar_users(1,model)
similar_idx = [x[0] for x in similar_item_list]
users_df.iloc[similar_idx][['user_id','first_name','last_name']]



,user_id,first_name,last_name
1,60b721a36dcadb0291ef7736,NaN,NaN
707,62af072a0c0205a124403a0c,Sd,sd
19,61326f4e2e44eb4d9b35e6d0,M,K


## Recommendation for new user (cold start problem)

In [117]:
new_user = pd.DataFrame(np.zeros(len(user_features_col))).T
new_user.columns = user_features_col
new_user['are_you_over_21'] = 1
new_user['allergies_flag'] = 1
new_user['gender'] = 1
new_user = csr_matrix(new_user)
scores_new_user = model.predict(user_ids = 0,item_ids = np.arange(interactions.shape[1]), user_features=new_user)
top_items_new_user = restaurent_df.iloc[np.argsort(-scores_new_user)]
top_items_new_user[0:5][['restaurant_name','restaurant_description','city']]

,restaurant_name,restaurant_description,city
36,Malaka Spice,Each meal that you have at Malaka Spice is a r...,Pune
201,Vibe Restaurant,"As the name suggests, the vibes of a Desi cult...",Bangalore Division
12,Rose Wood,Serving an extensive range of pure vegetarian ...,Bangalore Division
32,BURGER 11,Pune’s first midnight Gourmet burger delivery ...,Konkan Division
24,The Lotus Sutra - Hyatt Regency Pune,"City’s latest open-air venue, The Lotus Deck c...",Pune
